In [105]:
import dash
from dash import Dash, html, dcc, Input, Output
import pandas as pd
import plotly.express as px
import warnings
import dash_bootstrap_components as dbc
import plotly.graph_objs as go
warnings.filterwarnings("ignore", category=FutureWarning)

In [106]:
df = pd.read_csv('purine data new.csv')
df

,Foods,Average Purines in mg uric acid/100 g (Average)
0,Beef,117.17
1,Fish,199.61
2,Chicken,139.75
3,Pork,139.78
4,Sheep,182.00
5,Bean,54.00
6,Apple,14.00
7,Avocado,19.00
8,Tofu,68.00
9,Banana,57.00


In [107]:
user = pd.read_excel('gout_patient_data.xlsx')
user

,Date,Total Daily Purine(mg),Water(L),Exercise(min),Uric Acid (μmol/L),Weight(kg),User
0,2023-01-03,788,3.5,56,519.00,82.3,John
1,2023-01-04,720,2.9,50,532.00,81.8,John
2,2023-01-05,748,2.3,59,508.70,82.2,John
3,2023-01-06,812,2.0,53,538.60,81.4,John
4,2023-01-07,793,3.4,49,535.95,82.3,John
...,...,...,...,...,...,...,...
175,2023-06-27,734,2.9,44,460.90,76.1,John
176,2023-06-28,659,2.1,47,534.35,76.8,John
177,2023-06-29,665,2.1,48,543.95,75.9,John
178,2023-06-30,677,2.2,47,445.95,76.0,John


In [108]:
user['Date'] = pd.to_datetime(user['Date'])
user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180 entries, 0 to 179
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Date                    180 non-null    datetime64[ns]
 1   Total Daily Purine(mg)  180 non-null    int64         
 2   Water(L)                180 non-null    float64       
 3   Exercise(min)           180 non-null    int64         
 4   Uric Acid (μmol/L)      180 non-null    float64       
 5   Weight(kg)              180 non-null    float64       
 6   User                    180 non-null    object        
dtypes: datetime64[ns](1), float64(3), int64(2), object(1)
memory usage: 10.0+ KB


In [109]:
user['Month'] = pd.to_datetime(user['Date']).dt.month

In [110]:
new_user = pd.melt(user, id_vars=['Date','Month','User'], var_name='Categories', value_name='Value')
new_user.head()

,Date,Month,User,Categories,Value
0,2023-01-03,1,John,Total Daily Purine(mg),788.0
1,2023-01-04,1,John,Total Daily Purine(mg),720.0
2,2023-01-05,1,John,Total Daily Purine(mg),748.0
3,2023-01-06,1,John,Total Daily Purine(mg),812.0
4,2023-01-07,1,John,Total Daily Purine(mg),793.0


In [181]:
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

food_quantities = {food: 0 for food in df['Foods']}

food_intake = []

app.layout = html.Div(children=[
    # 左侧部分
    html.Div(children=[
        html.Nav(className="navbar navbar-expand-lg bg-primary",style={'background-color': '#73B3D9'}, **{"data-bs-theme": "dark"}, children=[
            html.Div(className="container-fluid", children=[
                html.A("Personal Health Index", className="navbar-brand",style={'font-size': '50px'})
            ])
        ]),
        
        html.Br(),
        
        html.Div([
        dbc.Row([
            dbc.Col([
                #html.H2('Personal Health Index'),
                dcc.DatePickerSingle(
                    id='date-picker',
                    date=user['Date'].max()
                ),
                html.Div(id='selected-data')
            ]),
            dbc.Col([
                dbc.Card([
                    dbc.CardBody([
                        html.H2('Health Tips', style={'font-size': '30px','color': '#0E367C'}),
                        html.Blockquote([
                            html.P('A regular life is the secret of health and longevity.', className='mb-0'),
                            html.Br(),
                            html.Figcaption([
                                'Balzac'
                            ], className='blockquote-footer')
                        ]),
                    ],
                    ),
                ])
        ], align="end"),
    ]),


        html.Br(),
        html.Br(),

        # 中间部分
        html.Div([
            # 折线图
            html.Div([
                html.H2('Change in Personal Health Index', style={'font-size': '30px','color': '#0E367C'}),
                dcc.Dropdown(
                    new_user['Categories'].unique(),
                    value = 'Total Daily Purine(mg)',
                    id = 'category'
                ),
                
                dcc.Graph(id = 'graph'),
                
                dcc.Slider(
                    new_user['Month'].min(),
                    new_user['Month'].max(),
                    step=None,
                    id='month_slider',
                    value=new_user['Month'].min(),
                    marks={str(month):str(month) for month in new_user['Month'].unique()},
                )
            ], style={'width': '75%', 'display': 'inline-block'}),
            
            html.Br(),
            html.Br(),
            html.Br(),
            
            html.Div([
                html.H2('What did you eat today?', style={'font-size': '30px','color': '#0E367C'}),
                html.Div([
                    'Food',
                    dcc.Dropdown(
                        id='food',
                        options=[{'label': food, 'value': food} for food in df['Foods']],
                        multi=True
                    ),
                    html.Br(),
                    'Total Intake(g)',
                    dcc.Input(id='food_weight', value=350, type='number')
                ]),
                html.Br(),
                html.Div(id='user_intake'),
                dcc.Graph(id='purines-bar-chart'),
                html.Button('Add', id='add_button', n_clicks=0)
            ], style={'width': '75%', 'display': 'inline-block'}),

        ]),
    
            
    html.Br(),
        
    # 显示新闻标题
    html.Div([
        dbc.Card(
            dbc.CardBody([
                 html.H4("Health News - About Hyperuricemia", className="card-title", style={'font-size': '30px'}),
                html.H6("Hyperuricemia: Symptoms, Treatment, and More", className="card-subtitle mb-2 text-muted"),
                html.P("Hyperuricemia occurs when there’s too much uric acid in your blood. Only about a third of people experience symptoms. If uric acid levels remain high, over time they can lead to several diseases, such as gout or kidney stones.", className="card-text"),
                dbc.CardLink("News link", href="https://www.healthline.com/health/hyperuricemia#symptoms"),
            ])
        ),
        
        html.Br(),
        
    html.Div([
        dbc.Card(
            dbc.CardBody([
                 html.H4("Health Artical - About Hyperuricemia", className="card-title", style={'font-size': '30px'}),
                html.H6("The treatment of hyperuricemia", className="card-subtitle mb-2 text-muted"),
                html.P("The present review article will try to summarize the most recent evidences on the efficacy of XO inhibitors and uricosuric compounds in lowering uric acid levels in both the bloodstream and peripheral tissues.", className="card-text"),
                dbc.CardLink("Artical link", href="https://www.sciencedirect.com/science/article/pii/S016752731530317X"),
            ]),
        ),
    ]),
]),
]),
]),
])

        

@app.callback(
    Output('selected-data', 'children'),
    Input('date-picker', 'date')
)
def update_selected_data(selected_date):
    filtered_df = user[user['Date'] == selected_date]

    if not filtered_df.empty:
        user_name = filtered_df.iloc[0]['User']
        water = filtered_df.iloc[0]['Water(L)']
        exercise = filtered_df.iloc[0]['Exercise(min)']
        uric_acid = filtered_df.iloc[0]['Uric Acid (μmol/L)']
        weight = filtered_df.iloc[0]['Weight(kg)']

        return html.Div([
            html.Div(f'{user_name}', style={'font-size': '50px',
                                            'font-weight': 'bold',
                                            'color': '#0E367C'}),
            html.Br(),
            dbc.Stack([
                html.Div(f'Water(L): {water}', style={'font-size': '19px'}),
                html.Div(f'Exercise(min): {exercise}', style={'font-size': '19px'}),
                html.Div(f'Uric Acid (μmol/L): {uric_acid}', style={'font-size': '19px'}),
                html.Div(f'Weight(kg): {weight}', style={'font-size': '19px'}),
            ],
            direction = 'horizontal',
            gap=3,
            ),
        ])
    else:
        return f'No data available for {selected_date}'

@app.callback(
    Output('graph', 'figure'),
    Input('category', 'value'),
    Input('month_slider', 'value')
    )
def update_graph(categories, month_value):
    query = (new_user['Month'] == month_value) & (new_user['Categories'] == categories)
    filtered_df = new_user[query]
    
    fig = px.line(
        data_frame = filtered_df,
        x = 'Date',
        y = 'Value',
        title = f'{categories} over Time')

    return fig

def calculate_total_purines_intake(df, food_quantities, intake_list):
    total_intake = 0 

    for food, quantity in food_quantities.items():
        if quantity is not None: 
            row = df[df['Foods'] == food]
            if not row.empty:
                purines_value = row.iloc[0]['Average Purines in mg uric acid/100 g (Average)']
                if not pd.isna(purines_value):
                    purines_intake = (purines_value / 100) * quantity
                    total_intake += purines_intake

                   
                    intake_list.append({'Food': food, 'Purines Intake': purines_intake})

    return total_intake, intake_list

@app.callback(
    Output('food', 'options'),
    Input('add_button', 'n_clicks')
)
def update_food_options(n_clicks):
    return [{'label': food, 'value': food} for food in food_quantities]

@app.callback(
    Output('user_intake', 'children'),
    Output('purines-bar-chart', 'figure'),
    Input('add_button', 'n_clicks'),
    Input('food', 'value'),
    Input('food_weight', 'value')
)
def update_user_intake(n_clicks, selected_food, weight):
    global food_intake
    food_intake = []  

    
    if not selected_food:
        return "No food selected", {"data": [], "layout": {"title": "Purines Intake by Food"}}

    if not isinstance(selected_food, list):
        selected_food = [selected_food]

    for food in food_quantities:
        if food in selected_food:
            food_quantities[food] = weight
        else:
            food_quantities[food] = 0

    total_purines_intake, food_intake_updated = calculate_total_purines_intake(df, food_quantities, food_intake)

    intake_df = pd.DataFrame(food_intake_updated)
    
    fig = go.Figure()
    fig.add_trace(go.Bar(x=intake_df['Food'], y=intake_df['Purines Intake'], name='Purines Intake'))
    fig.update_layout(
        title='Purines Intake by Food',
        xaxis_title='Food',
        yaxis_title='Purines Intake (mg)'
    )
    
    total_purines_intake_formatted = f'Total Purines Intake: {total_purines_intake:.2f} mg'
    
    return total_purines_intake_formatted, fig

if __name__ == '__main__':
    app.run_server(debug=True, port=8061)